### **Задание 1**   
Обучим нейронную сеть решать шифр Цезаря

*Что необходимо сделать:*

1. Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
2. Сделать нейронную сеть
3. Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
4. Проверить качество

### **Задание 2**   

1. Построить RNN-ячейку на основе полносвязных слоев
2. Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

#### 1. Написать алгоритм шифра Цезаря

In [1]:
# загрузим необходимые библиотеки
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import random
import string

# Определяем устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# Параметры
ALPHABET = string.ascii_lowercase + " "  # алфавит (буквы + пробел)
CHAR_TO_INDEX = {char: idx for idx, char in enumerate(ALPHABET)}  # символ → индекс
INDEX_TO_CHAR = {idx: char for char, idx in CHAR_TO_INDEX.items()}  # индекс → символ
VOCAB_SIZE = len(ALPHABET)  # размер словаря
MAX_LEN = 50  # максимальная длина фразы

In [3]:
# Функция для шифрования текста
def caesar_cipher(text, shift):
    result = []
    for char in text:
        if char in CHAR_TO_INDEX:
            idx = (CHAR_TO_INDEX[char] + shift) % VOCAB_SIZE
            result.append(INDEX_TO_CHAR[idx])
        else:
            result.append(char)  # если символ не в алфавите, оставляем как есть
    return "".join(result)

In [4]:
# Генерация данных
def generate_data(num_samples):
    data = []
    for _ in range(num_samples):
        # Генерируем случайную фразу длиной от 10 до MAX_LEN символов
        length = random.randint(10, MAX_LEN)
        text = "".join(random.choices(ALPHABET, k=length))
        shift = random.randint(1, 10)  # случайный сдвиг от 1 до 10
        encrypted_text = caesar_cipher(text, shift)  # шифруем текст
        data.append((encrypted_text, text, shift))  # сохраняем зашифрованный текст, исходный текст и сдвиг
    return data

#### 2. Построим нейронную сеть

1. Разбьем данные на токены (у нас символы)
2. Закодируем числами
3. Превратим в эмбеддинги

In [5]:
# Преобразование текста в тензор
def text_to_tensor(text, max_len):
    tensor = torch.zeros(max_len, dtype=torch.long).to(device)  # перемещаем на устройство
    for i, char in enumerate(text[:max_len]):
        tensor[i] = CHAR_TO_INDEX.get(char, CHAR_TO_INDEX[" "])  # неизвестные символы → пробел
    return tensor

In [6]:
HIDDEN_SIZE = 256  # размер скрытого состояния
EPOCHS = 30  # количество эпох
BATCH_SIZE = 128  # размер батча
LEARNING_RATE = 0.001  # learning rate
DROPOUT = 0.2  # Dropout

#### 3. Обучим нейронную сеть RNN
3 слоя:
1. Embeding (30)
2. RNN (hidden_dim=128)
3. Полносвязный слой для предсказания буквы (28, то есть размер словаря)

In [7]:
# Нейронная сеть с учетом сдвига
class CaesarDecoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout):
        super(CaesarDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.shift_embedding = nn.Embedding(11, hidden_size)  # сдвиги от 0 до 10
        self.rnn = nn.RNN(hidden_size * 2, hidden_size, batch_first=True, num_layers=2, dropout=dropout)  # вход: символы + сдвиг
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, shift):
        char_emb = self.embedding(x)  # эмбеддинги символов
        shift_emb = self.shift_embedding(shift).unsqueeze(1).expand(-1, x.size(1), -1)  # эмбеддинги сдвига
        combined = torch.cat((char_emb, shift_emb), dim=2)  # объединяем символы и сдвиг
        x, _ = self.rnn(combined)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [8]:
# Подготовка данных
data = generate_data(20000)  # генерируем 20,000 примеров
encrypted_texts, original_texts, shifts = zip(*data)

In [9]:
# Преобразуем тексты в тензоры
X = torch.stack([text_to_tensor(text, MAX_LEN) for text in encrypted_texts]).to(device)  # перемещаем на устройство
Y = torch.stack([text_to_tensor(text, MAX_LEN) for text in original_texts]).to(device)  # перемещаем на устройство
shifts = torch.tensor(shifts, dtype=torch.long).to(device)  # преобразуем сдвиги в тензор

In [10]:
# Создаем модель, функцию потерь и оптимизатор
model = CaesarDecoder(VOCAB_SIZE, HIDDEN_SIZE, DROPOUT).to(device)  # перемещаем модель на устройство
criterion = nn.CrossEntropyLoss().to(device)  # перемещаем функцию потерь на устройство
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)  # используем Adam
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # добавляем scheduler

Обучение:

In [11]:
# Обучение модели
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    total_correct = 0  # для подсчета правильных предсказаний
    total_samples = 0  # для подсчета общего числа символов

    for i in range(0, len(X), BATCH_SIZE):
        # Берем батч
        X_batch = X[i:i + BATCH_SIZE]
        Y_batch = Y[i:i + BATCH_SIZE]
        shift_batch = shifts[i:i + BATCH_SIZE]  # сдвиги для батча

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(X_batch, shift_batch)  # передаем сдвиг в модель
        outputs = outputs.view(-1, VOCAB_SIZE)
        Y_batch = Y_batch.view(-1)

        # Вычисляем потери
        loss = criterion(outputs, Y_batch)
        total_loss += loss.item()

        # Обратный проход и обновление весов
        loss.backward()
        optimizer.step()

        # Вычисляем accuracy
        _, predicted = torch.max(outputs, dim=1)  # получаем предсказанные символы
        total_correct += (predicted == Y_batch).sum().item()  # считаем правильные предсказания
        total_samples += Y_batch.size(0)  # общее количество символов в батче

    # Обновляем learning rate
    scheduler.step()

    # Выводим статистику
    accuracy = total_correct / total_samples  # вычисляем accuracy
    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / (len(X) // BATCH_SIZE):.4f}, Accuracy: {accuracy:.4f}")

Epoch 1/30, Loss: 0.7161, Accuracy: 0.8272
Epoch 2/30, Loss: 0.0458, Accuracy: 0.9795
Epoch 3/30, Loss: 0.0391, Accuracy: 0.9805
Epoch 4/30, Loss: 0.0362, Accuracy: 0.9824
Epoch 5/30, Loss: 0.0347, Accuracy: 0.9827
Epoch 6/30, Loss: 0.0344, Accuracy: 0.9824
Epoch 7/30, Loss: 0.0328, Accuracy: 0.9832
Epoch 8/30, Loss: 0.0314, Accuracy: 0.9839
Epoch 9/30, Loss: 0.0305, Accuracy: 0.9846
Epoch 10/30, Loss: 0.0300, Accuracy: 0.9847
Epoch 11/30, Loss: 0.0288, Accuracy: 0.9852
Epoch 12/30, Loss: 0.0284, Accuracy: 0.9855
Epoch 13/30, Loss: 0.0282, Accuracy: 0.9856
Epoch 14/30, Loss: 0.0282, Accuracy: 0.9857
Epoch 15/30, Loss: 0.0281, Accuracy: 0.9856
Epoch 16/30, Loss: 0.0281, Accuracy: 0.9857
Epoch 17/30, Loss: 0.0276, Accuracy: 0.9858
Epoch 18/30, Loss: 0.0278, Accuracy: 0.9859
Epoch 19/30, Loss: 0.0279, Accuracy: 0.9858
Epoch 20/30, Loss: 0.0277, Accuracy: 0.9860
Epoch 21/30, Loss: 0.0273, Accuracy: 0.9861
Epoch 22/30, Loss: 0.0272, Accuracy: 0.9862
Epoch 23/30, Loss: 0.0272, Accuracy: 0.98

#### 4. Проверим качество модели

In [12]:
# Проверка качества
def decode_text(model, encrypted_text, shift):
    model.eval()
    with torch.no_grad():
        # Ограничиваем длину входного текста
        input_length = len(encrypted_text)
        tensor = text_to_tensor(encrypted_text, input_length).unsqueeze(0).to(device)  # перемещаем на устройство
        shift_tensor = torch.tensor([shift], dtype=torch.long).to(device)  # преобразуем сдвиг в тензор
        output = model(tensor, shift_tensor)
        _, indices = torch.max(output, dim=2)
        decoded_text = "".join([INDEX_TO_CHAR[idx.item()] for idx in indices[0][:input_length]])  # обрезаем до длины исходного текста
    return decoded_text

In [13]:
# Пример использования
texts = [
    "hello world", "neural network", "caesar cipher", "deep learning",
    "machine learning", "artificial intelligence", "data science",
    "python programming", "pytorch framework", "encryption algorithms"
]

print("Testing the model on multiple examples:")
for text in texts:
    shift = random.randint(1, 10)  # случайный сдвиг от 1 до 10
    encrypted_text = caesar_cipher(text, shift)
    decoded_text = decode_text(model, encrypted_text, shift)
    print(f"Original: {text}")
    print(f"Shift: {shift}")
    print(f"Encrypted: {encrypted_text}")
    print(f"Decoded: {decoded_text}")
    print("-" * 40)

Testing the model on multiple examples:
Original: hello world
Shift: 7
Encrypted: olssvgcvysk
Decoded: hello world
----------------------------------------
Original: neural network
Shift: 6
Encrypted: tk xgrftkzbuxq
Decoded: neural network
----------------------------------------
Original: caesar cipher
Shift: 1
Encrypted: dbftbsadjqifs
Decoded: caesar cipher
----------------------------------------
Original: deep learning
Shift: 4
Encrypted: hiitdpievrmrk
Decoded: deep learning
----------------------------------------
Original: machine learning
Shift: 6
Encrypted: sginotkfrkgxtotm
Decoded: machine learning
----------------------------------------
Original: artificial intelligence
Shift: 8
Encrypted: izaqnqkqithqvamttqomvkm
Decoded: artificial intelligence
----------------------------------------
Original: data science
Shift: 7
Encrypted: kh hgzjplujl
Decoded: data science
----------------------------------------
Original: python programming
Shift: 3
Encrypted: sawkrqcsurjudpplqj
Decod

### Задание 2


#### 1. Построим RNN-ячейку на основе полносвязных слоев

In [15]:
# Загрузка данных
df = pd.read_csv('/content/simpsons_script_lines.csv')
phrases = df['normalized_text'].dropna().tolist()  # используем столбец normalized_text

<ipython-input-15-f4ab2aa3fa7b>:2: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/simpsons_script_lines.csv')


In [16]:
df.head()

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,True,40.0,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33


In [17]:
# Преобразуем тексты в список символов
text = [[char for char in phrase] for phrase in phrases if isinstance(phrase, str)]

In [18]:
# Создаем тензор для хранения индексов символов
X = torch.zeros((len(text), MAX_LEN), dtype=torch.long).to(device)  # перемещаем на устройство

# Заполняем тензор индексами символов
for i, phrase in enumerate(text):
    for j, char in enumerate(phrase):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(char, CHAR_TO_INDEX[" "])  # неизвестные символы → пробел

In [19]:
# Кастомная RNN-ячейка на основе полносвязных слоев
class CustomRNNCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CustomRNNCell, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size, hidden_size)  # вход -> скрытое состояние
        self.h2h = nn.Linear(hidden_size, hidden_size)  # скрытое состояние -> скрытое состояние

    def forward(self, x, hidden):
        hidden = torch.tanh(self.i2h(x) + self.h2h(hidden))  # обновляем скрытое состояние
        return hidden

In [20]:
# Нейронная сеть с кастомной RNN-ячейкой
class TextGenerator(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout):
        super(TextGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.rnn_cell = CustomRNNCell(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size

    def forward(self, x, hidden=None):
        if hidden is None:
            hidden = torch.zeros(x.size(0), self.hidden_size).to(device)  # инициализация скрытого состояния
        x = self.embedding(x)
        outputs = []
        for i in range(x.size(1)):  # проходим по каждому символу в последовательности
            hidden = self.rnn_cell(x[:, i, :], hidden)  # обновляем скрытое состояние
            outputs.append(hidden)
        outputs = torch.stack(outputs, dim=1)  # объединяем выходы
        outputs = self.dropout(outputs)
        outputs = self.fc(outputs)
        return outputs, hidden

In [21]:
# Подготовка данных
X = X.to(device)
Y = torch.roll(X, shifts=-1, dims=1).to(device)  # сдвигаем X на 1 влево для получения целевых символов

In [22]:
# Создаем модель, функцию потерь и оптимизатор
model = TextGenerator(VOCAB_SIZE, HIDDEN_SIZE, DROPOUT).to(device)  # перемещаем модель на устройство
criterion = nn.CrossEntropyLoss().to(device)  # перемещаем функцию потерь на устройство
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)  # используем Adam

In [24]:
# Обучение модели
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    total_correct = 0  # для подсчета правильных предсказаний
    total_samples = 0  # для подсчета общего числа символов

    for i in range(0, len(X), BATCH_SIZE):
        # Берем батч
        X_batch = X[i:i + BATCH_SIZE]
        Y_batch = Y[i:i + BATCH_SIZE]

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs, _ = model(X_batch)
        outputs = outputs.view(-1, VOCAB_SIZE)
        Y_batch = Y_batch.view(-1)

        # Вычисляем потери
        loss = criterion(outputs, Y_batch)
        total_loss += loss.item()

        # Обратный проход и обновление весов
        loss.backward()
        optimizer.step()

        # Вычисляем accuracy
        _, predicted = torch.max(outputs, dim=1)  # получаем предсказанные символы
        total_correct += (predicted == Y_batch).sum().item()  # считаем правильные предсказания
        total_samples += Y_batch.size(0)  # общее количество символов в батче

    # Выводим статистику
    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / (len(X) // BATCH_SIZE):.4f}")

Epoch 1/30, Loss: 1.2279
Epoch 2/30, Loss: 1.2115
Epoch 3/30, Loss: 1.1989
Epoch 4/30, Loss: 1.1910
Epoch 5/30, Loss: 1.1851
Epoch 6/30, Loss: 1.1816
Epoch 7/30, Loss: 1.1775
Epoch 8/30, Loss: 1.1749
Epoch 9/30, Loss: 1.1725
Epoch 10/30, Loss: 1.1708
Epoch 11/30, Loss: 1.1687
Epoch 12/30, Loss: 1.1671
Epoch 13/30, Loss: 1.1668
Epoch 14/30, Loss: 1.1651
Epoch 15/30, Loss: 1.1646
Epoch 16/30, Loss: 1.1640
Epoch 17/30, Loss: 1.1625
Epoch 18/30, Loss: 1.1622
Epoch 19/30, Loss: 1.1621
Epoch 20/30, Loss: 1.1617
Epoch 21/30, Loss: 1.1611
Epoch 22/30, Loss: 1.1597
Epoch 23/30, Loss: 1.1600
Epoch 24/30, Loss: 1.1603
Epoch 25/30, Loss: 1.1604
Epoch 26/30, Loss: 1.1594
Epoch 27/30, Loss: 1.1588
Epoch 28/30, Loss: 1.1585
Epoch 29/30, Loss: 1.1588
Epoch 30/30, Loss: 1.1595


#### 2. Применим построенную ячейку для генерации текста с выражениями героев сериала "Симпсоны"

In [25]:
# Генерация текста
def generate_text(model, start_text, max_length=50):
    model.eval()
    with torch.no_grad():
        # Преобразуем начальный текст в тензор
        input_tensor = text_to_tensor(start_text, len(start_text)).unsqueeze(0).to(device)
        hidden = None
        generated_text = list(start_text)

        for _ in range(max_length):
            output, hidden = model(input_tensor, hidden)
            _, predicted = torch.max(output[:, -1, :], dim=1)
            next_char = INDEX_TO_CHAR[predicted.item()]
            generated_text.append(next_char)
            input_tensor = text_to_tensor(next_char, 1).unsqueeze(0).to(device)

        return "".join(generated_text)

In [26]:
# Пример генерации текста
start_text = "homer"
generated_text = generate_text(model, start_text, max_length=50)
print(f"Generated text: {generated_text}")

Generated text: homer the see the show you the see the show you the see
